In [8]:
from activ import load_data, data_normalization
import os
from activ.readfile import TrackTBIFile
%matplotlib inline
import matplotlib.pyplot as plt
import h5py
import numpy as np
from activ.analytics import heatmap, nmf_bases_heatmap
from activ.clustering import UmapClusteringResults
from os.path import dirname, join
from matplotlib.backends.backend_pdf import PdfPages
from IPython.display import display, Latex, Markdown
from pylatex import Document, Section, Subsection, Command
from pylatex import Document, Section, Figure, SubFigure, NoEscape
from pylatex.utils import italic, NoEscape
from scipy.stats import norm, sem

In [9]:
raw = load_data()
labels = raw.outcome_features
nmfdata = load_data(uoinmf=True)
rawbm = raw.biomarkers #(247,235)
feature_names = raw.outcome_features
A = data_normalization.data_normalization(raw.outcomes, 'positive') #(247,177)
w = nmfdata.outcomes #(247,6)
h = nmfdata.outcome_bases #(6,177)

In [10]:
def drop_x_percent(A,w,h,feature_names,percent):
    total_features = h.shape[1]
    num_features_to_drop = np.round(total_features*percent//100)
    h_sum = np.sum(h, axis=0)
    h_sum_sorted = np.argsort(h_sum)
    indices_lowest_values = h_sum_sorted[:num_features_to_drop]
    dropped_features = sorted(feature_names[indices_lowest_values])
    h_new = np.delete(h, indices_lowest_values, axis=1)
    A_new = np.delete(A, indices_lowest_values, axis=1) 
    return A_new, h_new, dropped_features

In [11]:
percent = [0,10,20,30,40,50,60,70,80,90]
for p in percent:
    A_features_dropped,h_new, dropped_features = drop_x_percent(A,w,h,feature_names,p)
    fname = '/Users/ahyeon/data/activ/oc_features_dropped/oc_data_features_dropped_{}.h5'.format(p)
    TrackTBIFile.write(fname, rawbm, A_features_dropped)